# Computation for Proposition 3.2

In [1]:
import os
import pandas as pd
from inspect import getsource
from utils import *
if not os.path.exists('data/'):
    os.makedirs('data')

We shall compute the upper bound of $$h-3\log\text{rad}(N) + 4\log(2)$$ for $h\le e^{31}$.

In [2]:
if not os.path.isfile('data/list_of_f2_3e7.pkl'):
    list_of_f2_3e7 = list_of_f2(3*pow(10,7))
    pd.to_pickle(list_of_f2_3e7,"data/list_of_f2_3e7.pkl")
else:
    list_of_f2_3e7 = pd.read_pickle("data/list_of_f2_3e7.pkl")

my_primes = []
for p in primes(3*pow(10,7)):
    if p >= 23 and p not in (37,43,67,163):
        my_primes.append(p)
N = len(my_primes)

100%|██████████| 90000001/90000001 [00:40<00:00, 2202319.75it/s]


Let $p_0 < \cdots < p_{N-1}$ the the primes in the list *my_primes*.
For each $0\le t \le N-1$, when $p_t \cdot p_{t+1} > h / \log(2)$,
put $k = 2$, $S=\{p_t, \dots, p_{t+n-1}\}$ for some choice of $2 \le n = |S| \le N+1-t$,
then we can apply Lemma 3.1 $(ii)$ to get the inequality
$$ h \le (3+a_1)\cdot \log\text{rad}(N) + a_2 + \frac{2h}{n},$$
hence [by multiplying $3$, adding $a_1\cdot h$, and dividing $3+a_1$, we have]
$$ h \le 3 \cdot \log\text{rad}(N) + \frac{a_1+6n^{-1}}{3+a_1}\cdot h +\frac{3a_2}{3+a_1}.$$


Let $h_1 = p_s*p_{s+1}*\log(2)$, $1 < h_0 < h_1$. 
Then for any $h_0\le h < h_1$, we have 
$$h - 3\cdot \log\text{rad}(N) + 4\log(2) \le a_h\cdot \sqrt{h\log(h)},$$
where we can take $$a_h = \frac{a_1+6n^{-1}}{3+a_1}\cdot \sqrt{\frac{h_1}{\log(h_1)}}
+(\frac{3a_2}{3+a_1}+4\log(2) )\cdot \sqrt{\frac{1}{h_0\log(h_0)}}.$$

The following code verifies the inequality
$$h - 3\cdot \log\text{rad}(N) + 4\log(2) \le 8 \cdot \sqrt{h\log(h)}$$
for $350 \le h \le e^{31}$.

In [3]:
list_of_diff_info = []
choice_of_t = [0]
tmp = 0
for t in range(500000):
    if t - tmp < t/20 and t>100:
        continue
    choice_of_t.append(t)
    tmp = t

h0 = 0
h1 = 100
for t in tqdm(choice_of_t):
    p0 = my_primes[t]
    p1 = my_primes[t+1]
    n = int(3 / 2 * p0 / log(p0)-4)
    if t + n > N:
        print("Error!")
    a1 = 0 # avg of O(1/p)
    a2 = 0 # avg of log-volume
    for i in range(n):
        p = my_primes[t+i]
        a1 += (11*p*+31)/(p*p*p+p*p-12*p)
        a2 += list_of_f2_3e7[p] # 
    a1 = a1 / n/2
    a2 = 3 * a2 / n
    h0 = h1
    h1 = p0*p1*log(2)
    if h0 > 1000:
        hl, hr = h0, h1
        diff = (a1+6/n)/(3+a1)*sqrt(hr/log(hr)) + (a2+4*log(2))/sqrt(hl*log(hl))
        list_of_diff_info.append([hl,hr,diff])
    else:
        for i in range(100):
            hl = h0 + (h1-h0)*i/100
            hr = h0 + (h1-h0)*(i+1)/100
            diff = (a1+6/n)/(3+a1)*sqrt(hr/log(hr)) + (a2+4*log(2))/sqrt(hl*log(hl))
            if hr < 350:
                continue
            list_of_diff_info.append([hl,hr,diff])


print(list_of_diff_info)

100%|██████████| 266/266 [00:08<00:00, 32.91it/s]  

[[346.3838352147688, 350.0071269091036, 7.807641695254738], [350.0071269091036, 353.63041860343844, 7.785618066248023], [353.63041860343844, 357.2537102977733, 7.764073068187856], [357.2537102977733, 360.87700199210815, 7.742993291777138], [360.87700199210815, 364.50029368644294, 7.722365828147046], [364.50029368644294, 368.1235853807778, 7.7021782454572225], [368.1235853807778, 371.74687707511265, 7.682418566808835], [371.74687707511265, 375.3701687694475, 7.663075249384821], [375.3701687694475, 378.9934604637823, 7.644137164737881], [378.9934604637823, 382.61675215811715, 7.625593580152749], [382.61675215811715, 386.24004385245195, 7.6074341410145525], [386.24004385245195, 389.8633355467868, 7.5896488541200195], [389.8633355467868, 393.48662724112165, 7.572228071872846], [393.48662724112165, 397.1099189354565, 7.555162477308658], [397.1099189354565, 400.7332106297913, 7.538443069898855], [400.7332106297913, 404.35650232412615, 7.5220611520861596], [404.35650232412615, 407.97979401846

In [4]:
# check whether $h\le e^{31}$ is covered
print("Is the case $h\le e^\{31\}$ covered? ",log(list_of_diff_info[-1][1])>31)
# check that $\alpha_h < 8$
flag = True
for diff_info in list_of_diff_info:
    if diff_info[2]  > 7.9:
        flag = False
        print(diff)
print(flag)

Is the case $h\le e^\{31\}$ covered?  True
True
